# Model Tuning / Model Doğrulama

In [9]:
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import model_selection
import matplotlib.pyplot as plt

In [10]:
hit = pd.read_csv("Hitters.csv")

df = hit.copy()

df = df.dropna()

dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])

y = df["Salary"]

X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis = 1).astype("float64")

X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

enet_model = ElasticNet().fit(X_train, y_train)

y_pred_test = enet_model.predict(X_test)

C:\Users\Sefa3\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8487984.799367279, tolerance: 3898.686956380658
  positive)


In [11]:
np.sqrt(mean_squared_error(y_test, y_pred_test))

357.1676548181244

In [12]:
r2_score(y_test, y_pred_test)

0.41070222469326945

In [13]:
enet_cv_model = ElasticNetCV(cv = 10, random_state = 0)

In [14]:
enet_cv_model.fit(X_train, y_train)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=None,
             normalize=False, positive=False, precompute='auto', random_state=0,
             selection='cyclic', tol=0.0001, verbose=0)

In [15]:
enet_cv_model.alpha_

5230.7647364798695

In [16]:
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_)

In [17]:
enet_tuned.fit(X_train, y_train)

ElasticNet(alpha=5230.7647364798695, copy_X=True, fit_intercept=True,
           l1_ratio=0.5, max_iter=1000, normalize=False, positive=False,
           precompute=False, random_state=None, selection='cyclic', tol=0.0001,
           warm_start=False)

## test seti için final modelimiz;

In [18]:
y_pred_test = enet_tuned.predict(X_test)

In [19]:
np.sqrt(mean_squared_error(y_test, y_pred_test))

394.15280563218795

------

In [21]:
# "alphas" parametresi için bir liste oluşturmadık ve bunu modele bıraktık.
# ElasticNetCV fonksiyonu uygun olan değeri buldu.
# Eğer oluşturulmak istenirse alphas = [., ., .,] şeklinde bir liste oluşturulabilir.
# ve ElasticNetCV içerisine argüman olarak ifade edilebilir.

In [22]:
# "l1_ratio" parametresi
# Bu parametre değeri 0 olduğunda L2 cezalandırması,
# değeri 1 olduğunda L1 cezalandırması yapar.
# Dolayısıyla 0-1 arasındaki değişimi aslında cezalandırma metodlarının göreceli
# etkilerini ifade eder.
# Uygulamamızda ikisininde etkisinin aynı düzeyde olmasını istediğimizden dolayı
# bunun ön tanımlı değerini olduğu gibi bıraktık : yani 0.5

In [24]:
# "l1_ratio" ve "alphas" beraber ölçülürse
# "l1_ratio" için aranacak bir liste verildiğinde yani ceza türlerinin
# etkilerini ayarlamak için bir liste verildiğinde her bir "l1_ratio" değeri için
# farklı "alphas" değerleri olacaktır ki bu durumda bu alphalar ceza terimleri
# etkilerini modele yansıtmış olacaktır.

#### elimizdeki VERİ SETİ İÇİN en iyi çalışan modelin Lasso Regresyon Modeli olduğu gözlenmiştir.

Ridge Regresyon için bulunan hata : 386.21231390701064

Lasso Regresyon için bulunan hata : 356.5226376958367

ElasticNet Regresyon için bulunan hata : 394.15280563218795

olarak tespit edilmiştir.

## En düşük hata 'Lasso Regresyonunda'